# **Corporate Policies and the Term Structure of Risk**

*Matthijs Breugem, Roberto Marfè, Francesca Zucchi*

This notebook solves a coupled system of nonlinear second-order ODEs that describe corporate financial policies under uncertainty about the **term structure of risk prices**. The model allows the firm’s manager to potentially **misperceive** the market prices of long- and short-term risk, leading to suboptimal decisions.

We distinguish between:

- **$w(c)$**: the **manager’s perceived scaled firm value**, used to compute the optimal investment policy $l(c)$
- **$v(c)$**: the **true scaled firm value**, conditional on the firm following investment policy $l(c)$ derived from $w(c)$

Here, $c = M / A$ is the firm’s cash-to-asset ratio.


### 1. Economic Derivation of the Coupled ODE System

The system stems from the firm’s intertemporal optimization problem under uncertainty and adjustment costs. Letting $V(a, m)$ denote firm value as a function of assets $a$ and cash $m$, the Bellman equation implies:

$$
r V = \text{(expected change in value)}
$$

Exploiting scale invariance, the problem reduces to one state: the cash-to-asset ratio $c = m / a$. We define the **scaled value function** $v(c)$ such that $V(a, m) = a \cdot v(c)$. Because of stochastic shocks, the HJB equation becomes a **second-order nonlinear ODE** in $v(c)$. The optimal investment policy $l(c)$ solves a first-order condition that links marginal and average value:

$$
l(c) = \frac{1}{\kappa} \left( \frac{v(c)}{v'(c)} - (1 + c) \right)
$$

If the firm **misperceives** the term structure of risk (i.e., uses incorrect prices of long- and short-term risk), it forms expectations based on a distorted value function $w(c)$. The policy $l(c)$ is then computed from $w(c)$, while the true value $v(c)$ still evolves under that same $l(c)$. This leads to a **coupled ODE system**: one equation for $w(c)$ (subjective beliefs), one for $v(c)$ (true valuation), both tied by the same policy rule $l(c)$.


### 2. Investment Policy $l(c)$

The perceived optimal investment rate is:

$$
l(c) = -\frac{1 + c - \frac{w(c)}{w'(c)}}{\kappa}
$$


### 3. Coupled ODE System

Given $u = (w, w', v, v')$, the coupled system is:

$$
\begin{aligned}
w''(c) &= \frac{2(r - \mu + \eta_{Aw} \rho_A \sigma_A - l(c))w(c) + \text{Coeff}_w(c) \cdot w'(c)}{\sigma_A^2 c^2 + \sigma_Y^2} \\
v''(c) &= \frac{2(r - \mu + \eta_A \rho_A \sigma_A - l(c))v(c) + \text{Coeff}_v(c) \cdot v'(c)}{\sigma_A^2 c^2 + \sigma_Y^2}
\end{aligned}
$$

with:

$$
\begin{aligned}
\text{Coeff}_w(c) &= -2\left[\alpha + c(r - \lambda - \mu + \eta_{Aw} \rho_A \sigma_A) - \eta_{Yw} \rho_Y \sigma_Y \right] + 2(1 + c)l(c) + \kappa l(c)^2 \\
\text{Coeff}_v(c) &= -2\left[\alpha + c(r - \lambda - \mu + \eta_A \rho_A \sigma_A) - \eta_Y \rho_Y \sigma_Y \right] + 2(1 + c)l(c) + \kappa l(c)^2
\end{aligned}
$$


### 4. Boundary Conditions

We solve the system over $c \in [0, C_{\max}]$, imposing four boundary conditions:

- At $c = C_{\max}$:

$$
v'(C_{\max}) = 1, \quad v''(C_{\max}) = 0
$$

- At $c = C_{\min}$:

$$
v'(C_{\min}) = 1 + p, \quad v(0) = v(C_{\min}) - C_{\min}(1 + p) - f
$$


### 5. Benchmark: Flat Term Structure

In the special case where the manager has **correct beliefs** about risk pricing:

$$
\eta_A = \eta_{Aw}, \quad \eta_Y = \eta_{Yw}
$$

then $w(c) = v(c)$, and the two ODEs coincide. This benchmark is useful to evaluate the distortions caused by **misperceived pricing slopes**.


In [1]:
using DifferentialEquations, NLsolve, Random, Statistics, Dates, Plots, Interpolations

In [2]:
# 1. Define parameters (15 total)
σA, σY, κ = 0.24, 0.20, 3.0
r, λ, μ = 0.045, 0.02, -0.15
α, ρA, ρY = 0.25, 0.4, 0.2
pval, f = 0.05, 0.005
ηA, ηY = 0.5, 0.1         # for v(c)
ηAw, ηYw = 0.3, 0.3       # for w(c)

# Unified parameter tuple
params = (σA, σY, κ, r, λ, μ, α, ρA, ρY, pval, f, ηA, ηY, ηAw, ηYw)

(0.24, 0.2, 3.0, 0.045, 0.02, -0.15, 0.25, 0.4, 0.2, 0.05, 0.005, 0.5, 0.1, 0.3, 0.3)

In [3]:
# 2. Coupled ODE system for w(c) and v(c)
function coupled_ode!(du, u, p, c)
    # Unpack state variables
    w, dw, v, dv = u

    # Unpack parameters
    (σA, σY, κ, r, λ, μ, α, ρA, ρY, pval, f, ηA, ηY, ηAw, ηYw) = p

    # Compute ℓ(c) from w and w′
    safe_dw = sign(dw) * max(abs(dw), 1e-6)
    ℓ = -(1 + c - w / safe_dw) / κ

    # Denominator shared by both ODEs
    denom = σA^2 * c^2 + σY^2

    # Coefficients for w''
    term1_w = 2 * (r - μ + ηAw * ρA * σA - ℓ) * w
    coeff_w = -2 * (α + c * (r - λ - μ + ηAw * ρA * σA) - ηYw * ρY * σY) + 2 * (1 + c) * ℓ + κ * ℓ^2
    term2_w = coeff_w * dw

    # Coefficients for v''
    term1_v = 2 * (r - μ + ηA * ρA * σA - ℓ) * v
    coeff_v = -2 * (α + c * (r - λ - μ + ηA * ρA * σA) - ηY * ρY * σY) + 2 * (1 + c) * ℓ + κ * ℓ^2
    term2_v = coeff_v * dv

    # ODE system
    du[1] = dw
    du[2] = (term1_w + term2_w) / denom
    du[3] = dv
    du[4] = (term1_v + term2_v) / denom
end


coupled_ode! (generic function with 1 method)

In [4]:
# 3. Define full system incl. BC's
function residuals_coupled!(F::Vector{Float64}, x::Vector{Float64}, p)
    CMINw, CMAXw, wCMAX, CMINv, CMAXv, vCMAX = x
    (σA, σY, κ, r, λ, μ, α, ρA, ρY, pval, f, ηA, ηY, ηAw, ηYw) = p

    # Initial values: slopes fixed at CMAX
    u0 = [wCMAX, 1.0, vCMAX, 1.0]
    cmax = maximum((CMAXw, CMAXv))
    prob = ODEProblem(coupled_ode!, u0, (cmax, 0.0), p)
    sol = solve(prob, Tsit5(); reltol=1e-8, abstol=1e-10)

    # Extract solution at key points
    wCMIN, dwCMIN = sol(CMINw)[1:2]
    vCMIN, dvCMIN = sol(CMINv)[3:4]

    w0 = sol(0.0)[1]
    v0 = sol(0.0)[3]

    # Estimate curvature at CMAX
    eps = 1e-6
    d2wCMAX = (sol(CMAXw + eps)[2] - sol(CMAXw - eps)[2]) / (2 * eps)
    d2vCMAX = (sol(CMAXv + eps)[4] - sol(CMAXv - eps)[4]) / (2 * eps)

    # six residuals
    F[1] = d2wCMAX                                  # w″(CMAXw) = 0
    F[2] = dwCMIN - (1 + pval)                      # w′(CMINw) = 1 + p
    F[3] = w0 - (wCMIN - CMINw * (1 + pval) - f)        # w(0) condition

    F[4] = d2vCMAX                                  # v″(CMAXv) = 0
    F[5] = dvCMIN - (1 + pval)                      # v′(CMINv) = 1 + p
    F[6] = v0 - (vCMIN - CMINv * (1 + pval) - f)        # v(0) condition
end


residuals_coupled! (generic function with 1 method)

In [5]:
x0 = [0.05, 0.17, 1.0, 0.05, 0.19, 1.0]
sol = nlsolve((F, x) -> residuals_coupled!(F, x, params), x0; method = :trust_region)


Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.05, 0.17, 1.0, 0.05, 0.19, 1.0]
 * Zero: [0.08670944452417692, 0.28128952537035773, 1.3207617581090183, 0.08291013251249213, 0.2682763760119111, 1.269783520184801]
 * Inf-norm of residuals: 0.000000
 * Iterations: 35
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 34
 * Jacobian Calls (df/dx): 19

-------------------

In [6]:
using NLsolve, DifferentialEquations, Random, LinearAlgebra

# --- PARAMETER BOUNDS ---
lower = (0.20, 0.10, 2.8, 0.04, 0.02, -0.20, 0.25, 0.20, 0.05, 0.04, 0.001)
upper = (0.30, 0.30, 3.2, 0.06, 0.05, -0.15, 0.30, 0.50, 0.50, 0.08, 0.005)

# --- Agent-specific eta values ---
etas_RA = (0.1, 0.5, 0.1, 0.5)   # Rational Agent
etas_IS = (0.1, 0.5, 0.3, 0.3)   # Ignoring Slope Agent

# --- Initial guess for the solver ---
x0 = [0.05, 0.28, 1.0, 0.05, 0.28, 1.0]

# --- Percent difference helper ---
function pct_diff(x, y)
    return 100 * (y - x) / x
end

pct_diff (generic function with 1 method)

In [7]:
function simulate_one(p_rand)
    outputs = Dict()

    for (label, etas) in [("RA", etas_RA), ("IS", etas_IS)]
        p = (p_rand..., etas...)
        sol = nlsolve((F, x) -> residuals_coupled!(F, x, p), x0; method = :trust_region)

        if !sol.f_converged
            return nothing
        end

        CMINw, CMAXw, wCMAX, CMINv, CMAXv, vCMAX = sol.zero
        u0_w = [wCMAX, 1.0, 0.0, 0.0]
        prob_w = ODEProblem(coupled_ode!, u0_w, (CMAXw, 0.0), p)
        sol_w = solve(prob_w, Tsit5(); reltol=1e-8, abstol=1e-10)

        κ = p[3]
        function ℓ_at(c)
            w, dw = sol_w(c)[1:2]
            safe_dw = sign(dw) * max(abs(dw), 1e-6)
            return -(1 + c - w / safe_dw) / κ
        end

        ℓ0 = ℓ_at(0.0)
        ℓC = ℓ_at(CMAXv)

        if max(ℓ0, ℓC) < 0
            return nothing
        end

        ℓ_interp = LinearInterpolation(
            range(0.0, CMAXw; length=300),
            [ℓ_at(c) for c in range(0.0, CMAXw; length=300)],
            extrapolation_bc=Line()
        )

        function ode_v!(du, u, p_, c)
            v, dv = u
            (σA, σY, κ, r, λ, μ, α, ρA, ρY, pval, f, ηA, ηY, ηAw, ηYw) = p_
            ℓ = ℓ_interp(c)
            safe_dv = sign(dv) * max(abs(dv), 1e-6)
            term1 = 2 * (r - μ + ηA * ρA * σA - ℓ) * v
            coeff = -2 * (α + c * (r - λ - μ + ηA * ρA * σA) - ηY * ρY * σY) +
                    2 * (1 + c) * ℓ + κ * ℓ^2
            denom = σA^2 * c^2 + σY^2
            d2v = (term1 + coeff * dv) / denom
            du[1] = dv
            du[2] = d2v
        end

        u0_v = [vCMAX, 1.0]
        prob_v = ODEProblem(ode_v!, u0_v, (CMAXv, 0.0), p)
        sol_v = solve(prob_v, Tsit5(), reltol=1e-8, abstol=1e-10)

        v0 = sol_v(0.0)[1]
        vC = sol_v(CMAXv)[1]

        outputs[label] = (v0, vC, ℓ0, ℓC)
    end

    if haskey(outputs, "RA") && haskey(outputs, "IS")
        v0_RA, vC_RA, ℓ0_RA, ℓC_RA = outputs["RA"]
        v0_IS, vC_IS, ℓ0_IS, ℓC_IS = outputs["IS"]

        return (
            v0_RA, v0_IS, pct_diff(v0_RA, v0_IS),
            vC_RA, vC_IS, pct_diff(vC_RA, vC_IS),
            ℓ0_RA, ℓ0_IS, pct_diff(ℓ0_RA, ℓ0_IS),
            ℓC_RA, ℓC_IS, pct_diff(ℓC_RA, ℓC_IS)
        )
    end

    return nothing
end


simulate_one (generic function with 1 method)

In [8]:
# Generate one random parameter draw within bounds
p_rand = ntuple(j -> lower[j] + rand() * (upper[j] - lower[j]), 11)
println("🔍 Trying p_rand = $p_rand")

# Run the simulation
result = simulate_one(p_rand)

# Check result
if result === nothing
    println("❌ Simulation discarded (non-converged or ℓ condition failed)")
else
    println("✅ Simulation accepted!")
    println("Result = ")
    println(result)
end


🔍 Trying p_rand = (0.29710954451602356, 0.22353225344751648, 2.855744672232349, 0.05541205437347443, 0.021416354509466957, -0.17024538081295007, 0.29801562827675243, 0.2478031730606365, 0.2139277871077514, 0.07131270799218721, 0.0028301723223861505)
✅ Simulation accepted!
Result = 
(1.1554369567817948, 1.1538805835461963, -0.13469997012502197, 1.4914540971530659, 1.4890779546357045, -0.15931717388400063, -0.003434894965253194, 0.06843908106899478, -2092.4650320115384, 0.05872089519298778, 0.049252033348082404, -16.125200090675918)


In [9]:
successful = 0
total = 2000

elapsed_time = @elapsed begin
    for _ in 1:total
        p_rand = ntuple(j -> lower[j] + rand() * (upper[j] - lower[j]), 11)
        result = simulate_one(p_rand)
        if result !== nothing
            successful += 1
        end
    end
end

println("🎯 Summary: $successful / $total simulations accepted.")
println("⏱️  Total time: $(round(elapsed_time; digits=2)) seconds")


┌ Warning: At t=-1.6756246172116587, dt was forced below floating point epsilon 2.220446049250313e-16, and step error estimate = NaN. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/c6Noy/src/integrator_interface.jl:623
┌ Warning: At t=-0.3544972704538491, dt was forced below floating point epsilon 5.551115123125783e-17, and step error estimate = NaN. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/c6Noy/src/integrator_interface.jl:623


🎯 Summary: 1051 / 2000 simulations accepted.
⏱️  Total time: 34.3 seconds
